In [18]:
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install pillow
!pip install keras

In [131]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Directory paths
input_dir = 'input_images'
target_dir = 'target_images'
intermediate_dir = 'intermediate_images'

In [132]:
# Function to load images from a directory
def load_images(image_dir):
    images = []
    for filename in sorted(os.listdir(image_dir)):
        if filename.endswith('.png'):
            img_path = os.path.join(image_dir, filename)
            img = load_img(img_path, target_size=(512, 512))
            img = img_to_array(img)
            img = (img / 127.5) - 1  # Normalize to [-1, 1]
            images.append(img)
    return np.array(images)

# Load input and target images
input_images = load_images(input_dir)
target_images = load_images(target_dir)

# Load intermediate images
intermediate_images = []
for subdir in sorted(os.listdir(intermediate_dir)):
    intermediate_subdir = os.path.join(intermediate_dir, subdir)
    if os.path.isdir(intermediate_subdir):
        images = load_images(intermediate_subdir)
        intermediate_images.append(images)

# Create TensorFlow datasets
input_dataset = tf.data.Dataset.from_tensor_slices(input_images)
target_dataset = tf.data.Dataset.from_tensor_slices(target_images)

# Convert intermediate images list to tuple of datasets
intermediate_datasets = tuple(tf.data.Dataset.from_tensor_slices(steps) for steps in intermediate_images)

# Combine into one dataset
dataset = tf.data.Dataset.zip((input_dataset, intermediate_datasets, target_dataset))

# Shuffle and batch the dataset
batch_size = 32
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)

# Split into training and validation sets
total_samples = len(input_images) + sum(len(steps) for steps in intermediate_images)
train_size = int(0.8 * total_samples)

# Split into training and validation sets
val_size = total_samples - train_size
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)

# Print dataset information
# Print dataset information
print("Number of training samples:", train_size)
print("Number of validation samples:", val_size)

Number of training samples: 4
Number of validation samples: 1


In [133]:
# Print lengths of input, intermediate, and target images
print("Number of input images:", len(input_images))
print("Number of intermediate images folders:", len(intermediate_images))
print("Number of target images:", len(target_images))

# Print number of images in each intermediate folder
for i, steps in enumerate(intermediate_images):
    print(f"Number of images in intermediate folder {i+1}:", len(steps))


Number of input images: 1
Number of intermediate images folders: 1
Number of target images: 1
Number of images in intermediate folder 1: 4


In [149]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential

# Function to build the generator (U-Net architecture)
def build_generator(input_shape):
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
    conv2 = Conv2D(128, 4, strides=2, padding='same', activation='relu')(conv1)
    conv3 = Conv2D(256, 4, strides=2, padding='same', activation='relu')(conv2)
    conv4 = Conv2D(512, 4, strides=2, padding='same', activation='relu')(conv3)
    
    # Decoder
    deconv1 = Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu')(conv4)
    deconv1_concat = Concatenate()([deconv1, conv3])
    deconv2 = Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu')(deconv1_concat)
    deconv2_concat = Concatenate()([deconv2, conv2])
    deconv3 = Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu')(deconv2_concat)
    deconv3_concat = Concatenate()([deconv3, conv1])
    outputs = Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(deconv3_concat)
    
    return Model(inputs, outputs)

# Function to build the discriminator (PatchGAN architecture)
def build_discriminator(input_shape):
#     inputs = Input(shape=input_shape)
    
#     conv1 = Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
#     conv2 = Conv2D(128, 4, strides=2, padding='same', activation='relu')(conv1)
#     conv3 = Conv2D(256, 4, strides=2, padding='same', activation='relu')(conv2)
#     conv4 = Conv2D(512, 4, strides=2, padding='same', activation='relu')(conv3)
#     outputs = Conv2D(1, 4, strides=1, padding='same', activation='sigmoid')(conv4)
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))  # Output shape (batch_size, 1)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
    
    # return Model(inputs, outputs)

# Define input shape
input_shape = (512, 512, 3)

# Build generator and discriminator
generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)

# Display model summaries
generator.summary()
discriminator.summary()



/Users/joudat/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_47"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_80 (Conv2D)  │ (None, 256, 256,  │      3,136 │ input_layer_24[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_81 (Conv2D)  │ (None, 128, 128,  │    131,200 │ conv2d_80[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_82 (Conv2D)  │ (None, 64, 64,    │    524,544 │ conv2d_81[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_83 (Conv2D)  │ (None, 32, 32,    │  2,097,664 │ conv2d_82[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_36 │ (None, 64, 64,    │  2,097,408 │ conv2d_83[0][0]   │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_27      │ (None, 64, 64,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 512)              │            │ conv2d_82[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_37 │ (None, 128, 128,  │  1,048,704 │ concatenate_27[0… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_28      │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_81[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_38 │ (None, 256, 256,  │    262,208 │ concatenate_28[0… │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_29      │ (None, 256, 256,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 128)              │            │ conv2d_80[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_39 │ (None, 512, 512,  │      6,147 │ concatenate_29[0… │
│ (Conv2DTranspose)   │ 3)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,171,011 (23.54 MB)

 Trainable params: 6,171,011 (23.54 MB)

...

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_84 (Conv2D)              │ (None, 510, 510, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8323200)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │     8,323,201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,324,097 (31.75 MB)

 Trainable params: 8,324,097 (31.75 MB)

 Non-trainable params: 0 (0.00 B)

In [150]:
import keras
from keras import layers
from keras.optimizers import Adam

# Define optimizer for both generator and discriminator
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# Compile discriminator
discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

# Compile generator
generator.compile(optimizer=optimizer, loss='binary_crossentropy', run_eagerly=True)

# Define GAN model
discriminator.trainable = False  # Set discriminator to non-trainable
gan_input = Input(shape=input_shape)
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = Model(gan_input, gan_output)

# Compile GAN
gan.compile(optimizer=optimizer, loss='binary_crossentropy', run_eagerly=True)


In [158]:
import numpy as np

# Define number of epochs and batch size
epochs = 10
batch_size = 1

# Define function to generate batches of real images
def generate_real_samples(dataset, batch_size):
    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)
    # Get a batch of images and labels
    iterator = iter(dataset)
    real_images_tuple = next(iterator)

    # Print to understand the structure
    print("Type of real_images_tuple:", type(real_images_tuple))
    
    # Check if real_images_tuple contains the actual tensors
    if isinstance(real_images_tuple, tuple):
        for idx, item in enumerate(real_images_tuple):
            print(f"Index {idx} in real_images_tuple:", item)
    
    # Assume the actual images are in the first element of the tuple
    real_images = real_images_tuple[0]

    print("Shape of real_images before reshaping:", real_images.shape)

    # If the shape is (1, 3, 512, 512, 3), reshape to (batch_size, 512, 512, 3)
    if real_images.shape == (1, 1, 512, 512, 3):
        real_images = tf.reshape(real_images, (batch_size, 512, 512, 3))
    else:
        # Print and debug if shape is different
        print("Unexpected shape:", real_images.shape)
    
    print("Shape of real_images after reshaping:", real_images.shape)

    labels = tf.ones((batch_size, 1))  # Label real images as 1 (real)
    return real_images, labels


In [152]:
# Example usage
# Assuming dataset is defined elsewhere and properly structured
# dataset = ...
batch_size = 1
real_images, labels = generate_real_samples(dataset, batch_size)


Type of real_images_tuple: <class 'tuple'>
Index 0 in real_images_tuple: tf.Tensor(
[[[[[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   ...

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]]]], shape=(1, 1, 512, 512, 3), dtype=float32)
Index 1 in real_images_tuple: (<tf.Tensor: shape=(1, 1, 512, 512, 3), dtype=float32, numpy=
array([[[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          ...,
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [

In [153]:
# Define function to train the discriminator

def train_discriminator(discriminator, generator, dataset, batch_size):
    # Generate real samples
    real_images, real_labels = generate_real_samples(dataset, batch_size)

    # Generate fake samples
    noise = np.random.randn(batch_size, 512, 512, 3)
    fake_images = generator.predict(noise)
    fake_labels = np.zeros((batch_size, 1))  # Label fake images as 0 (fake)

    # Train discriminator on real and fake samples
    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    print(f'Discriminator loss: {d_loss}')
    return d_loss


In [154]:
# Define function to train the generator
def train_generator(gan_model, batch_size):
    noise = np.random.randn(batch_size, 512, 512, 3)
    labels = np.ones((batch_size, 1))  # Label generated images as 1 (real)
    g_loss = gan_model.train_on_batch(noise, labels)
    
    print(f'Generator loss: {g_loss}')
    return g_loss

In [156]:
import matplotlib.pyplot as plt

def save_generated_images(generator, epoch, batch, noise_dim=(512, 512, 3), examples=3):
    noise = np.random.randn(examples, *noise_dim)
    generated_images = generator.predict(noise)
    
    for i, img in enumerate(generated_images):
        plt.subplot(1, examples, i+1)
        plt.imshow((img * 127.5 + 127.5).astype(np.uint8))  # Assuming generator output is in range [-1, 1]
        plt.axis('off')
    
    plt.savefig(f'generated_images_epoch{epoch}_batch{batch}.png')
    plt.close()

In [159]:

# Main training loop
for epoch in range(epochs):
    # print(len(dataset))
    # print(batch_size)
    # print(len(dataset) // batch_size)
    for batch in range(len(dataset) // batch_size):
        # Train discriminator
        d_loss = train_discriminator(discriminator, generator, dataset, batch_size)

        # Train generator (via GAN model)
        g_loss = train_generator(gan, batch_size)

        # Print loss after each batch
        # print(f'Epoch {epoch + 1}/{epochs} - Batch {batch + 1}/{len(dataset) // batch_size} - Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')
        # print(f'Epoch {epoch + 1}/{epochs} - Batch {batch + 1}/{len(input_dataset) // batch_size} - Discriminator Loss: {d_loss}, Generator Loss: {g_loss}')
        print(f'Epoch {epoch + 1}/{epochs} - Batch {batch + 1}/{len(input_dataset) // batch_size} - Discriminator Loss: {d_loss}, Generator Loss: {g_loss}')
        
        # Save generated images
        save_generated_images(generator, epoch + 1, batch + 1)
        
    # Optionally, monitor training progress after each epoch
    # print(f'Epoch {epoch + 1}/{epochs} - Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')

Type of real_images_tuple: <class 'tuple'>
Index 0 in real_images_tuple: tf.Tensor(
[[[[[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   ...

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]

   [[1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]
    ...
    [1. 1. 1.]
    [1. 1. 1.]
    [1. 1. 1.]]]]], shape=(1, 1, 512, 512, 3), dtype=float32)
Index 1 in real_images_tuple: (<tf.Tensor: shape=(1, 1, 512, 512, 3), dtype=float32, numpy=
array([[[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          ...,
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [